# 0. 준비

## 1. 각종 필요 기능 import

In [1]:
# 데이터 처리 관련 준비
import pandas as pd
import numpy as np

# 크롤링 관련 준비
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

# 대기를 위한 WebDriverWait import
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC # 기다리는 조건을 뭘로 할 지 정해줌

# 값을 기다릴 때마다 WebDriverWait을 호출해야하므로 함수로 정의해버리자.
def wait_until(xpath_str):
    WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.XPATH, xpath_str)))

## 2. 전처리 된 데이터 불러온 후, 검색 컬럼 생성

In [2]:
df = pd.read_csv('D:/DataSet/서울시_일반음식점_인허가_정보/gangnam_general_restaurant.csv', encoding = 'utf-8')
df.drop(columns = '번호', inplace = True)
df

,사업장명,소재지전화,소재지전체주소,도로명전체주소,업태구분명,좌표정보(x),좌표정보(y),Unnamed: 47
0,바다드림,NaN,서울특별시 강동구 성내동 427-47,"서울특별시 강동구 풍성로54길 33, 1층 (성내동)",횟집,211730.041860,447312.111904,NaN
1,청년피자 고덕상일점,NaN,서울특별시 강동구 고덕동 193-4,"서울특별시 강동구 상일로15길 22, 1층 (고덕동)",경양식,214946.249742,450680.438962,NaN
2,빰빠라,NaN,서울특별시 강동구 길동 388-3,"서울특별시 강동구 양재대로111길 18, 2층 (길동)",라이브카페,212172.882863,448356.930270,NaN
3,오우,NaN,서울특별시 중구 주교동 289-1,서울특별시 중구 을지로27길 32-16 (주교동),기타,199910.827092,451776.715598,NaN
4,금덕푸드,NaN,서울특별시 중구 을지로6가 17-2,"서울특별시 중구 장충단로13길 20, 지하2층 (을지로6가)",기타,200613.510298,451817.515367,NaN
...,...,...,...,...,...,...,...,...
124665,온기정,053 255 7949,서울특별시 강남구 삼성동 159-7 현대백화점,"서울특별시 강남구 테헤란로 517, 현대백화점 지하1층 (삼성동)",일식,205210.358779,445154.422252,NaN
124666,놀부 공유주방 선릉역,NaN,서울특별시 강남구 대치동 896-28,"서울특별시 강남구 테헤란로68길 25, 지하1층 지하1호 (대치동)",한식,204603.136055,444606.621961,NaN
124667,인생갈비탕,NaN,서울특별시 강남구 역삼동 707-34 한신인터밸리24빌딩,"서울특별시 강남구 테헤란로 322, 한신인터밸리24빌딩 지하2층 B130호 (역삼동)",한식,204044.789949,444529.840427,NaN
124668,꾸어가게생선구이화곡점,0264089282,서울특별시 강서구 화곡동 827-2 1층,"서울특별시 강서구 등촌로13다길 35, 1층 (화곡동)",한식,187499.286366,448069.583478,NaN


In [3]:
df2 = pd.read_csv('D:/DataSet/서울시_일반음식점_인허가_정보/gangnam_general_restaurant.csv', encoding = 'utf-8')
df2.drop(columns = '번호', inplace = True)
df2

,사업장명,소재지전화,소재지전체주소,도로명전체주소,업태구분명,좌표정보(x),좌표정보(y),Unnamed: 47
0,바다드림,NaN,서울특별시 강동구 성내동 427-47,"서울특별시 강동구 풍성로54길 33, 1층 (성내동)",횟집,211730.041860,447312.111904,NaN
1,청년피자 고덕상일점,NaN,서울특별시 강동구 고덕동 193-4,"서울특별시 강동구 상일로15길 22, 1층 (고덕동)",경양식,214946.249742,450680.438962,NaN
2,빰빠라,NaN,서울특별시 강동구 길동 388-3,"서울특별시 강동구 양재대로111길 18, 2층 (길동)",라이브카페,212172.882863,448356.930270,NaN
3,오우,NaN,서울특별시 중구 주교동 289-1,서울특별시 중구 을지로27길 32-16 (주교동),기타,199910.827092,451776.715598,NaN
4,금덕푸드,NaN,서울특별시 중구 을지로6가 17-2,"서울특별시 중구 장충단로13길 20, 지하2층 (을지로6가)",기타,200613.510298,451817.515367,NaN
...,...,...,...,...,...,...,...,...
124665,온기정,053 255 7949,서울특별시 강남구 삼성동 159-7 현대백화점,"서울특별시 강남구 테헤란로 517, 현대백화점 지하1층 (삼성동)",일식,205210.358779,445154.422252,NaN
124666,놀부 공유주방 선릉역,NaN,서울특별시 강남구 대치동 896-28,"서울특별시 강남구 테헤란로68길 25, 지하1층 지하1호 (대치동)",한식,204603.136055,444606.621961,NaN
124667,인생갈비탕,NaN,서울특별시 강남구 역삼동 707-34 한신인터밸리24빌딩,"서울특별시 강남구 테헤란로 322, 한신인터밸리24빌딩 지하2층 B130호 (역삼동)",한식,204044.789949,444529.840427,NaN
124668,꾸어가게생선구이화곡점,0264089282,서울특별시 강서구 화곡동 827-2 1층,"서울특별시 강서구 등촌로13다길 35, 1층 (화곡동)",한식,187499.286366,448069.583478,NaN


In [4]:
df['naver_keyword'] = '강남%20' + df['사업장명']
df

,사업장명,소재지전화,소재지전체주소,도로명전체주소,업태구분명,좌표정보(x),좌표정보(y),Unnamed: 47,naver_keyword
0,바다드림,NaN,서울특별시 강동구 성내동 427-47,"서울특별시 강동구 풍성로54길 33, 1층 (성내동)",횟집,211730.041860,447312.111904,NaN,강남%20바다드림
1,청년피자 고덕상일점,NaN,서울특별시 강동구 고덕동 193-4,"서울특별시 강동구 상일로15길 22, 1층 (고덕동)",경양식,214946.249742,450680.438962,NaN,강남%20청년피자 고덕상일점
2,빰빠라,NaN,서울특별시 강동구 길동 388-3,"서울특별시 강동구 양재대로111길 18, 2층 (길동)",라이브카페,212172.882863,448356.930270,NaN,강남%20빰빠라
3,오우,NaN,서울특별시 중구 주교동 289-1,서울특별시 중구 을지로27길 32-16 (주교동),기타,199910.827092,451776.715598,NaN,강남%20오우
4,금덕푸드,NaN,서울특별시 중구 을지로6가 17-2,"서울특별시 중구 장충단로13길 20, 지하2층 (을지로6가)",기타,200613.510298,451817.515367,NaN,강남%20금덕푸드
...,...,...,...,...,...,...,...,...,...
124665,온기정,053 255 7949,서울특별시 강남구 삼성동 159-7 현대백화점,"서울특별시 강남구 테헤란로 517, 현대백화점 지하1층 (삼성동)",일식,205210.358779,445154.422252,NaN,강남%20온기정
124666,놀부 공유주방 선릉역,NaN,서울특별시 강남구 대치동 896-28,"서울특별시 강남구 테헤란로68길 25, 지하1층 지하1호 (대치동)",한식,204603.136055,444606.621961,NaN,강남%20놀부 공유주방 선릉역
124667,인생갈비탕,NaN,서울특별시 강남구 역삼동 707-34 한신인터밸리24빌딩,"서울특별시 강남구 테헤란로 322, 한신인터밸리24빌딩 지하2층 B130호 (역삼동)",한식,204044.789949,444529.840427,NaN,강남%20인생갈비탕
124668,꾸어가게생선구이화곡점,0264089282,서울특별시 강서구 화곡동 827-2 1층,"서울특별시 강서구 등촌로13다길 35, 1층 (화곡동)",한식,187499.286366,448069.583478,NaN,강남%20꾸어가게생선구이화곡점


In [5]:
df = df.head()
df

,사업장명,소재지전화,소재지전체주소,도로명전체주소,업태구분명,좌표정보(x),좌표정보(y),Unnamed: 47,naver_keyword
0,바다드림,NaN,서울특별시 강동구 성내동 427-47,"서울특별시 강동구 풍성로54길 33, 1층 (성내동)",횟집,211730.041860,447312.111904,NaN,강남%20바다드림
1,청년피자 고덕상일점,NaN,서울특별시 강동구 고덕동 193-4,"서울특별시 강동구 상일로15길 22, 1층 (고덕동)",경양식,214946.249742,450680.438962,NaN,강남%20청년피자 고덕상일점
2,빰빠라,NaN,서울특별시 강동구 길동 388-3,"서울특별시 강동구 양재대로111길 18, 2층 (길동)",라이브카페,212172.882863,448356.930270,NaN,강남%20빰빠라
3,오우,NaN,서울특별시 중구 주교동 289-1,서울특별시 중구 을지로27길 32-16 (주교동),기타,199910.827092,451776.715598,NaN,강남%20오우
4,금덕푸드,NaN,서울특별시 중구 을지로6가 17-2,"서울특별시 중구 장충단로13길 20, 지하2층 (을지로6가)",기타,200613.510298,451817.515367,NaN,강남%20금덕푸드


In [6]:
df.drop(index = 1, inplace = True)
df.drop(index = 2, inplace = True)
df

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23204\3250840991.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(index = 1, inplace = True)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23204\3250840991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(index = 2, inplace = True)


,사업장명,소재지전화,소재지전체주소,도로명전체주소,업태구분명,좌표정보(x),좌표정보(y),Unnamed: 47,naver_keyword
0,바다드림,NaN,서울특별시 강동구 성내동 427-47,"서울특별시 강동구 풍성로54길 33, 1층 (성내동)",횟집,211730.041860,447312.111904,NaN,강남%20바다드림
3,오우,NaN,서울특별시 중구 주교동 289-1,서울특별시 중구 을지로27길 32-16 (주교동),기타,199910.827092,451776.715598,NaN,강남%20오우
4,금덕푸드,NaN,서울특별시 중구 을지로6가 17-2,"서울특별시 중구 장충단로13길 20, 지하2층 (을지로6가)",기타,200613.510298,451817.515367,NaN,강남%20금덕푸드


# 1. 크롤링

In [19]:
# 크롤링 데이터를 담을 데이터 프레임 준비
c_columns = ['name','addresse','cat1', 'score', 'review']
data_frame = pd.DataFrame(columns = c_columns)

# 크롬 창 열기
browser = webdriver.Chrome(ChromeDriverManager().install()) # 크롬드라이버 자동 설치

# 별점과 메뉴, 리뷰 끌어오기
for i, keyword in enumerate(df['naver_keyword'].tolist()):

    print(keyword)
    
    # naver_keyword 순서대로 불러와서 검색
    naver_map_search_url = f"https://map.naver.com/v5/search/{keyword}"
    browser.get(naver_map_search_url)
    time.sleep(3)

    # 음식점 페이지가 한 번에 떴을 때
    try:
        browser.switch_to.default_content()
        iframe_1 = browser.find_element(By.XPATH, '//*[@id="entryIframe"]')
        browser.switch_to.frame(iframe_1)

        #사업장명
        name =  browser.find_element(By.XPATH, '//*[@id="_title"]/span[1]').text
        # 사업장 주소
        address = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/ul/li[2]/div/a/span[1]').text
        # 음식 종류
        if browser.find_element(By.XPATH, '//*[@id="_title"]/span[2]').is_displayed :
            cat1 =  browser.find_element(By.XPATH, '//*[@id="_title"]/span[2]').text

        # 별점이 존재한다면..?
        try:
            # 별점 추가
            score = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/em').text
        except:
            # 없으면 결측값
            score = np.nan

    except:
        # 검색결과가 리스트로 나올 때
        iframe_2 = browser.find_element(By.XPATH, '//*[@id="searchIframe"]')
        browser.switch_to.frame(iframe_2)
        time.sleep(2)
        

        try:
            browser.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[1]/div[2]/a[1]').click()
            time.sleep(2)

            iframe_3 = browser.find_element(By.XPATH,'/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-layout/div[2]/entry-layout/entry-place-bridge/div/nm-external-frame-bridge/nm-iframe/iframe')
            browser.switch_to.frame(iframe_3)
            time.sleep(2)

            #사업장명
            name =  browser.find_element(By.XPATH, '//*[@id="_title"]/span[1]').text
            # 사업장 주소
            address = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/ul/li[2]/div/a/span[1]').text
            # 음식 종류
            if browser.find_element(By.XPATH, '//*[@id="_title"]/span[2]').is_displayed :
                cat1 =  browser.find_element(By.XPATH, '//*[@id="_title"]/span[2]').text

            # 별점이 존재한다면..?
            try:
                # 별점 추가
                score = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/em').text
            except:
                # 없으면 결측값
                score = np.nan

        except:
            name = np.nan
            score = np.nan
            cat1 = np.nan
            address = np.nan

    print("데이터는 : ", name, score, cat1, address)

browser.quit()



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\xilverh0ya\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23204\1958724890.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install()) # 크롬드라이버 자동 설치


강남%20바다드림
데이터는 :  바다드림 4.33 생선회 서울 강동구 풍성로54길 33 1층
강남%20오우
데이터는 :  nan nan nan nan
강남%20금덕푸드
데이터는 :  nan nan nan nan


In [34]:
# 크롤링 데이터를 담을 데이터 프레임 준비
c_columns = ['name','addresse','cat1', 'cat2', 'score', 'review']
data_frame = pd.DataFrame(columns = c_columns)

# 크롬 창 열기
browser = webdriver.Chrome(ChromeDriverManager().install()) # 크롬드라이버 자동 설치

# 별점과 메뉴, 리뷰 끌어오기
for i, keyword in enumerate(df['naver_keyword'].tolist()):

    print(keyword)
    
    # naver_keyword 순서대로 불러와서 검색
    naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
    browser.get(naver_map_search_url)
    time.sleep(3)

    try:
        browser.find_element(By.XPATH,'//*[@id="ct"]/div[2]/ul/li/div[1]/a[2]/div/strong').click()
        time.sleep(2)

        #사업장명
        name =  browser.find_element(By.XPATH, '//*[@id="_title"]/span[1]').text
        # 사업장 주소
        address = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/ul/li[1]/div/a/span[1]').text
        # 음식 종류
        cat1 =  browser.find_element(By.XPATH, '//*[@id="_title"]/span[2]').text
        # 별점이 존재한다면..?
        try:
            # 별점 추가
            score = browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/em').text
        except:
            # 없으면 결측값
            score = np.nan
        
        # 방문자 리뷰 페이지 접속
        browser.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a').click()
        time.sleep(3)

        browser.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[7]/div[2]/div[1]/div/div/div[2]/a/svg').click()
        time.sleep(2)

        visitor_review = []
        

        # 메뉴 정보
        cat2 = []
        menus = browser.find_elements(By.CLASS_NAME, "info_menu")

        for menu in menus:
            cat2.append(menu.text)

    except:
        name = np.nan
        score = np.nan
        cat1 = np.nan
        address = np.nan

    print("데이터는 : ", name, score, cat1, address)

browser.quit()



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\xilverh0ya\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_23204\4227550139.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install()) # 크롬드라이버 자동 설치


강남%20바다드림
데이터는 :  nan nan nan nan
강남%20오우
데이터는 :  nan nan nan nan
강남%20금덕푸드
데이터는 :  nan nan nan nan


In [ ]:
driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe') 
map_url = "https://m.map.naver.com/search2/search.naver?query="
driver.get(map_url)

for i in range(len(df_db)):
    time.sleep(1)
    driver.find_element_by_xpath('''//*[@id="nx_query"]''').clear()
    keyword = driver.find_element_by_xpath('''//*[@id="nx_query"]''')
    keyword.send_keys("부산 ",dong[i]," ",name[i])
    keyword.send_keys(Keys.ENTER)
# 
    try:
        driver.find_element_by_xpath('//*[@id="loc-main-section-root"]/div/div[4]/ul/li[1]/div[1]/a[1]/div/div/span[1]/mark').click()
        time.sleep(1)
    except:
        try:
            driver.find_element_by_xpath('//*[@id="_title"]/span[1]').click()
            time.sleep(1)
        except:
            try:
                driver.find_element_by_xpath('//*[@id="place-main-section-root"]/div/div[5]/ul/li[1]/div[1]/a/div[1]/div/span[1]').click()
                time.sleep(1)
            except:
                star.append('NULL')
                review.append('NULL')
                blog.append('NULL')
                continue
                    
    star_review = driver.find_elements_by_class_name('_1Y6hi')
    count = len(star_review)
    print('%d개' % count)            
    
    if count == 2:
        star.append('NULL')
        try:
            review.append(driver.find_element_by_xpath('''//*[@id="app-root"]/div/div[2]/div[1]/div/div/div[1]/div/span[1]/a/em''').text)
        except:
            review.append('NULL')
        try:
            blog.append(driver.find_element_by_xpath('''//*[@id="app-root"]/div/div[2]/div[1]/div/div/div[1]/div/span[2]/a/em''').text)
        except:
            blog.append('NULL')
            
    elif count == 3:
        try:
            star.append(driver.find_element_by_xpath('''//*[@id="app-root"]/div/div[2]/div[1]/div/div/div[1]/div/span[1]/em''').text)
        except:
            star.append('NULL')
        try:
            review.append(driver.find_element_by_xpath('''//*[@id="app-root"]/div/div[2]/div[1]/div/div/div[1]/div/span[2]/a/em''').text)
        except:
            review.append('NULL')
        try:
            blog.append(driver.find_element_by_xpath('''//*[@id="app-root"]/div/div[2]/div[1]/div/div/div[1]/div/span[3]/a/em''').text)
        except:
            blog.append('NULL')
    else:
        star.append('NULL')
        review.append('NULL')
        blog.append('NULL')
        
    driver.back()

    
driver.close()
driver.quit()